<a href="https://colab.research.google.com/github/ShaelinN/Basketball/blob/main/COMP721_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
root = "/content/drive/MyDrive/COMP721_Project"

In [3]:
data = os.path.join(root, "Data")

#TEAM

In [109]:
team_season = pd.read_csv(os.path.join(data, "team_season.csv"))
team_season.head()

,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_3pa,o_pts,d_fgm,d_fga,d_ftm,d_fta,d_oreb,d_dreb,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
0,BOS,1946,N,1397,5133,811,1375,0,0,0,470,1202,0,0,0,0,0,3605,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3900,0.0,22,38
1,CH1,1946,N,1879,6309,939,1550,0,0,0,436,1473,0,0,0,0,0,4697,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4471,0.0,39,22
2,CL1,1946,N,1674,5699,903,1428,0,0,0,494,1246,0,0,0,0,0,4251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4308,0.0,30,30
3,DE1,1946,N,1437,5843,923,1494,0,0,0,482,1351,0,0,0,0,0,3797,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3918,0.0,20,40
4,NYK,1946,N,1465,5255,951,1438,0,0,0,457,1218,0,0,0,0,0,3881,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3840,0.0,33,27


##preprocessing:


* keep only NBA items
* combine win and loss counts into a single ratio
* drop unneeded columns

In [110]:
team_season = team_season.where(team_season["leag"]=="N")
team_season = team_season.dropna()
team_season = team_season.reset_index().drop("index",1)

win_rate = team_season["won"] / (team_season["won"]+team_season["lost"])
team_season["win_rate"] = win_rate

team_season = team_season.drop("won",1).drop("lost",1).drop("leag",1)

#if pd shows some cells in a column as ints, the entire column is ints. casting is safe
team_season.loc[:,"year":"d_pts"]= team_season.loc[:,"year":"d_pts"].astype(np.int64)
team_season.head()

,team,year,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_3pa,o_pts,d_fgm,d_fga,d_ftm,d_fta,d_oreb,d_dreb,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,win_rate
0,BOS,1946,1397,5133,811,1375,0,0,0,470,1202,0,0,0,0,0,3605,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3900,0.0,0.366667
1,CH1,1946,1879,6309,939,1550,0,0,0,436,1473,0,0,0,0,0,4697,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4471,0.0,0.639344
2,CL1,1946,1674,5699,903,1428,0,0,0,494,1246,0,0,0,0,0,4251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4308,0.0,0.500000
3,DE1,1946,1437,5843,923,1494,0,0,0,482,1351,0,0,0,0,0,3797,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3918,0.0,0.333333
4,NYK,1946,1465,5255,951,1438,0,0,0,457,1218,0,0,0,0,0,3881,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3840,0.0,0.550000


* MinMaxScaling on the data so the values in each feature are between 0 and 1

In [217]:
ts_vectors = team_season.loc[:,"o_fgm":] 
ts_vectors = np.array(ts_vectors)  #a consequence of including pace is that all values in this array are cast to floats

"""
# this is the wanted effect from feature scaling
# OMMITED because applying MimMaxScaler.fit_transform directly onto ts_vectors has the exact same result

table = np.copy(ts_vectors)
table = table.transpose()
for i in range(len(table)):
  col = table[i]
  col = np.array([col])
  col = col.transpose()

  col = MinMaxScaler().fit_transform(col)

  col = col.transpose()
  col = col[0]
  table[i] = col
table = table.transpose()
ts_vectors =  table

print(ts_vectors[0])
"""

ts_vectors = MinMaxScaler().fit_transform(ts_vectors)
print(ts_vectors[0])


[0.1427149  0.34989066 0.03907638 0.12767781 0.         0.
 0.         0.10691557 0.17447917 0.         0.         0.
 0.         0.         0.1011027  0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.1153896
 0.         0.33439153]


* formulate keys in the form <team , year> (as string to allow hashing)
* make a lookup/ dictionary to associate a feature vector with each key

In [ ]:
ts_id = [ 
         str(team_season["team"][i]) +" "+ str(int(team_season["year"][i]) ) 
         for i in range(   len(team_season["team"]) ) 
        ]
ts_id

lookup = {ts_id[i] : ts_vectors[i]   for i in range(len(ts_id))}
lookup["BOS 1946"]
print([i for i in lookup["BOS 1946"]])

#PLAYER

In [ ]:
player_rs_career = pd.read_csv(os.path.join(data, "player_regular_season_career.csv"))
player_rs_career.head()

,ilkid,firstname,lastname,leag,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
0,ABDELAL01,Alaa,Abdelnaby,N,256,3200,1465,283,563,846,85,71,69,247,484,1236,620,321,225,3,0
1,ABDULKA01,Kareem,Abdul-jabbar,N,1560,57446,38387,2975,9394,17440,5660,1160,3189,2527,4657,28307,15837,9304,6712,18,1
2,ABDULMA01,Mahmo,Abdul-rauf,N,586,15633,8553,219,868,1087,2079,487,46,963,1107,7943,3514,1161,1051,1339,474
3,ABDULTA01,Tariq,Abdul-wahad,N,236,4808,1830,275,501,776,266,184,82,309,485,1726,720,529,372,76,18
4,ABDURSH01,Shareef,Abdur-rahim,N,672,24862,13338,1498,3976,5474,1847,718,556,1911,1845,10215,4789,4427,3614,477,146


In [ ]:
NBA_prsc = player_rs_career.where(player_rs_career["leag"]=="N")
NBA_prsc = NBA_prsc.dropna()

ABA_prsc = player_rs_career.where(player_rs_career["leag"]=="A")
ABA_prsc= ABA_prsc.dropna()

rem = player_rs_career.where(player_rs_career["leag"]!="A")
rem = rem.where(rem["leag"]!="N")
rem = rem.dropna()

In [ ]:
print(player_rs_career.shape)
print(NBA_prsc.shape)
print(ABA_prsc.shape)
print(rem.shape)

In [ ]:
NBA_prsc = NBA_prsc.drop("firstname",1)
NBA_prsc = NBA_prsc.drop("lastname",1)
NBA_prsc = NBA_prsc.drop("leag",1)
NBA_prsc = NBA_prsc.drop("ilkid",1)
NBA_prsc.head()

,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
0,256.0,3200.0,1465.0,283.0,563.0,846.0,85.0,71.0,69.0,247.0,484.0,1236.0,620.0,321.0,225.0,3.0,0.0
1,1560.0,57446.0,38387.0,2975.0,9394.0,17440.0,5660.0,1160.0,3189.0,2527.0,4657.0,28307.0,15837.0,9304.0,6712.0,18.0,1.0
2,586.0,15633.0,8553.0,219.0,868.0,1087.0,2079.0,487.0,46.0,963.0,1107.0,7943.0,3514.0,1161.0,1051.0,1339.0,474.0
3,236.0,4808.0,1830.0,275.0,501.0,776.0,266.0,184.0,82.0,309.0,485.0,1726.0,720.0,529.0,372.0,76.0,18.0
4,672.0,24862.0,13338.0,1498.0,3976.0,5474.0,1847.0,718.0,556.0,1911.0,1845.0,10215.0,4789.0,4427.0,3614.0,477.0,146.0


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
model = KMeans(verbose=True)
model.fit(NBA_prsc)